In [2]:
from IPython.display import display, HTML
display(HTML(
"""
<a target="_blank" href="https://colab.research.google.com/drive/1FaonpQWNSsNVpJOhPbBAoJeMYy2tB-ce?usp=sharing">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
"""
))

In [ ]:
!git clone https://github.com/openai/shap-e
%cd shap-e

In [ ]:
!pip install blobfile
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

100%|████████████████████████████████████████| 890M/890M [00:07<00:00, 118MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

By changing the batch_size, you can control how many 3D meshes will be generated

In [ ]:
batch_size = 1
guidance_scale = 15.0
prompt = "a green humain heart with red cancer"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
render_mode = 'nerf' # you can change this to 'stf'
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

In [ ]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    with open(f'hear_cancer_mesh_{i}.ply', 'wb') as f:
        decode_latent_mesh(xm, latent).tri_mesh().write_ply(f)

--------------------------------------------------------------------------------------

## Convert PLY file into STL
Since a lot of medical imaging analysis needs STL file instead of PLY, then you can use the code below to do the conversion.

In [ ]:
!pip install trimesh

In [ ]:
import trimesh

In [ ]:
# Load the PLY file
input_ply_file = 'input_file.ply'
mesh = trimesh.load_mesh(input_ply_file)

# Convert the mesh to an STL file
output_stl_file = 'output_file.stl'
mesh.export(output_stl_file, file_type='stl')